In [1]:
import math
from sklearn.preprocessing import StandardScaler, MinMaxScaler

import custom_scores
import CMAPSAuxFunctions
from tunableModel import TunableModel

from keras.models import Sequential, Model
from keras.layers import Dense, Input, Dropout, Reshape, Conv2D, Flatten, MaxPooling2D
from keras.optimizers import Adam
from keras.callbacks import LearningRateScheduler
from keras import backend as K

from scipy.optimize import differential_evolution

Using TensorFlow backend.


In [2]:
#Clear the previous tensorflow graph
K.clear_session()

FilterN = 10
FilterL = 10

def RULmodel_SN_1(input_shape):
    
    #Create a sequential model
    model = Sequential()
    
    #Add the layers for the model
    model.add(Dense(250, input_dim=input_shape, activation='relu', kernel_initializer='glorot_normal', name='fc1'))
    model.add(Dropout(0.6))
    model.add(Dense(50, activation='relu', kernel_initializer='glorot_normal', name='fc2'))
    #model.add(Dense(5, activation='relu', kernel_initializer='glorot_normal', name='fc3'))
    model.add(Dropout(0.2))
    #model.add(Dense(10, activation='relu', name='fc3'))
    #model.add(Dropout(0.5))
    #model.add(Dense(10, activation='tanh', name='fc4'))
    #model.add(Dropout(0.5))
    model.add(Dense(1, activation='linear', name='out'))
    
    #create a placeholder for the input
    #X_input = Input(shape=(input_shape))
    
    #Create the layers
    #X = Dense(100, activation='relu', name='fc1')(X_input)
    #X = Dense(100, activation='relu', name='fc2')(X)
    #X = Dense(1, activation='linear', name='out')(X)
    
    # Create model. This creates the Keras model instance, you'll use this instance to train/test the model.
    #model = Sequential(inputs = X_input, outputs = X, name='RUL')
    
    return model


def RULModel_CNN_1(input_shape):
    
    #Create a sequential model
    model = Sequential()
    
    print(input_shape)
    
    #Add the layers for the model
    model.add(Conv2D(1, (FilterN, FilterL), input_shape=input_shape, padding='same', kernel_initializer='glorot_normal', activation='relu', name='cl1'))
    model.add(Dropout(0.6))
    model.add(MaxPooling2D(pool_size=5, name='pl1'))
    model.add(Flatten())
    model.add(Dense(20, activation='relu', kernel_initializer='glorot_normal', name='fc1'))
    model.add(Dropout(0.2))
    model.add(Dense(1, activation='linear', name='out'))
    
    return model


def RULCNNModel(TW, FeatureN):
    
    input_layer = Input(shape=(TW, FeatureN))
    y = Reshape((TW, FeatureN, 1), input_shape=(TW, FeatureN, ),name = 'Reshape')(input_layer)

    y = Conv2D(FilterN, FilterL, 1, border_mode='same', kernel_initializer='glorot_normal', activation='tanh', name='C1')(y)
    y = Conv2D(FilterN, FilterL, 1, border_mode='same', kernel_initializer='glorot_normal', activation='tanh', name='C2')(y)
    y = Conv2D(FilterN, FilterL, 1, border_mode='same', kernel_initializer='glorot_normal', activation='tanh', name='C3')(y)
    y = Conv2D(FilterN, FilterL, 1, border_mode='same', kernel_initializer='glorot_normal', activation='tanh', name='C4')(y)
    #y = Convolution2D(FilterN, FilterL, 1, border_mode='same', init='glorot_normal', activation='tanh', name='C5')(y)
    #y = Convolution2D(FilterN, FilterL, 1, border_mode='same', init='glorot_normal', activation='tanh', name='C6')(y)
    
    y = Conv2D(1, 3, 1, border_mode='same', kernel_initializer='glorot_normal', activation='tanh', name='Clast')(y)  
    
    y = Reshape((TW,14))(y)
    y = Flatten()(y)
    y = Dropout(0.5)(y)
    
    #y = Dense(100, activation='tanh', init='glorot_normal', activity_regularizer=keras.regularizers.l2(0.01),)(y)
    y = Dense(100,activation='tanh', kernel_initializer='glorot_normal', name='fc')(y)
    y = Dense(1)(y)
    
    model = Model(inputs = input_layer, outputs = y, name='RUL_CNN_Model')
    
    return model

In [4]:
#Shared parameters for the models
optimizer = Adam(lr=0, beta_1=0.5)
lossFunction = "mean_squared_error"
metrics = ["mse"]

#Create and compile the models
nFeatures = len(selected_features)
shapeSN = nFeatures*tModel.windowSize
shapeCNN1 = (tModel.windowSize, nFeatures, 1)
shapeDCNN1 = (tModel.windowSize, nFeatures)

modelRULSN = RULmodel_SN_1(shapeSN)
modelRULCNN = RULModel_CNN_1(shapeCNN1)
DCNN = RULCNNModel(shapeDCNN1[0], shapeDCNN1[1])

modelRULSN.compile(optimizer = optimizer, loss = lossFunction, metrics = metrics)
modelRULCNN.compile(optimizer = optimizer, loss = lossFunction, metrics = metrics)
DCNN.compile(optimizer = optimizer, loss = lossFunction, metrics = metrics)

#Add the models to the tunableModel object
#tModel.addModel('RULmodel_SN_1', modelRULSN)
#tModel.addModel('RULModel_CNN_1', modelRULCNN)
#tModel.addModel('RULCNNModel', DCNN)

NameError: name 'selected_features' is not defined

In [3]:
#Shared parameters for the models
optimizer = Adam(lr=0, beta_1=0.5)
lossFunction = "mean_squared_error"
metrics = ["mse"]

#Selected as per CNN paper
selected_features = ['T24', 'T30', 'T50', 'P30', 'Nf', 'Nc', 'Ps30', 'phi', 'NRf', 'NRc', 
                     'BPR', 'htBleed', 'W31', 'W32']

windowSize = 30
dataFolder = '../CMAPSSData'

#Create and compile the models
nFeatures = len(selected_features)
shapeSN = nFeatures*windowSize
modelRULSN = RULmodel_SN_1(shapeSN)
modelRULSN.compile(optimizer = optimizer, loss = lossFunction, metrics = metrics)

min_max_scaler = MinMaxScaler(feature_range=(-1, 1))

tModel = TunableModel('ModelRUL_SN_1', modelRULSN, selected_features, dataFolder, 
                      scaler = min_max_scaler, window_stride=1)
tModel.loadData(verbose=1)

Loading data for dataset 1 with window_size of 30, stride of 1 and constRUL of 125. Cros-Validation ratio 0
Data loaded for dataset 1


In [4]:
#tModel.printData(printTop=False)
#tModel.windowSize = 20
#tModel.changeDataset(2)
#tModel.loadData(verbose=1, crossValRatio=0.1)
#tModel.printData(printTop=False)
tModel.getModelDescription()

Printing shapes

Training data (X, y)
(17731, 420)
(17731, 1)
Testing data (X, y)
(100, 420)
(100, 1)
Printing last 5 elements

Training data (X, y)
[[ 0.62199313  0.32071611  0.36067504 ...,  0.45454545 -0.4488189
  -0.77474791]
 [ 0.20962199  0.14475703  0.60204082 ...,  0.09090909 -0.7480315
  -0.24669791]
 [ 0.12714777  0.14475703  0.71428571 ...,  0.63636364 -0.52755906
  -0.88893623]
 [ 0.03092784  0.14475703  0.78296703 ...,  0.09090909 -0.76377953
  -0.51768215]
 [ 0.26460481  0.15191816  0.50039246 ...,  0.27272727 -0.63779528
  -0.55120011]]
[[ 4.]
 [ 3.]
 [ 2.]
 [ 1.]
 [ 0.]]
Testing data (X, y)
[[-0.49140893 -0.54373402 -0.47959184 ..., -0.63636364  0.29133858
   0.60886238]
 [ 0.11340206 -0.19028133 -0.21350078 ...,  0.09090909 -0.25984252
   0.1393268 ]
 [-0.03780069  0.10537084 -0.18210361 ..., -0.09090909 -0.02362205
   0.32537992]
 [-0.32646048 -0.05012788 -0.49568289 ..., -0.63636364  0.27559055
   0.32168726]
 [-0.20962199  0.03375959  0.02943485 ...,  0.27272727 -0.

In [5]:
lrate = LearningRateScheduler(CMAPSAuxFunctions.step_decay)

#print(tModel.currentModelName)
#tModel.getAllModelsDescription()
#tModel.getModelDescription('RULModel_CNN_1')
#tModel.getModelDescription(tModel.currentModelName)
#tModel.getModelDescription()
#tModel.setCurrentModel('RULmodel_SN_1')
#tModel.getModelDescription()
tModel.epochs = 20
tModel.trainModel(learningRateScheduler=lrate, verbose=1)


Epoch 1/20
14184/14184 [==============================] - 1s 76us/step - loss: 4679.1091 - mean_squared_error: 4679.1091
Epoch 2/20
14184/14184 [==============================] - 0s 34us/step - loss: 757.7337 - mean_squared_error: 757.7337
Epoch 3/20
14184/14184 [==============================] - ETA: 0s - loss: 540.2549 - mean_squared_error: 540.25 - 0s 31us/step - loss: 538.2157 - mean_squared_error: 538.2157
Epoch 4/20
14184/14184 [==============================] - 0s 32us/step - loss: 468.1235 - mean_squared_error: 468.1235
Epoch 5/20
14184/14184 [==============================] - 1s 48us/step - loss: 422.6891 - mean_squared_error: 422.6891
Epoch 6/20
14184/14184 [==============================] - 0s 31us/step - loss: 395.6182 - mean_squared_error: 395.6182
Epoch 7/20
14184/14184 [==============================] - 1s 36us/step - loss: 383.0616 - mean_squared_error: 383.0616
Epoch 8/20
14184/14184 [==============================] - 0s 30us/step - loss: 370.9658 - mean_squared_error:

In [8]:
tModel.evaluateModel(["rhs"], crossValidation=True)
print("scores")

cScores = tModel.scores
rmse = math.sqrt(cScores['score_1'])
rhs = cScores['rhs']
time = tModel.trainTime

print(cScores)
print("RMSE: {}".format(rmse))
print("RHS: {}".format(rhs))
print("Time : {} seconds".format(time))

3547/3547 [==============================] - 0s 56us/step
scores
{'loss': 199.46597253042844, 'score_1': 199.46597253042844, 'rhs': 11209.523105929726}
RMSE: 14.12324228109213
RHS: 11209.523105929726
Time : 23.014435 seconds


In [ ]:
def nn_optmize_fun(x, selected_features=['T24', 'T30', 'T50', 'P30', 'Nf', 'Nc', 'Ps30', 'phi', 'NRf', 'NRc', 
                     'BPR', 'htBleed', 'W31', 'W32'], datasetNumber = '1', scaler = None, verbose=0, epochs=250, 
                  saveToFile = None, iterations = 0):
    
    #Clear the previous tensorflow graph
    K.clear_session()
    
    #Extract the tunning variables from the input vector
    #Round the values to the nearest integer since this implementation is for real numbers
    x = x.astype(int)
    windows_size = x[0]
    window_stride = x[1]
    constantRUL = x[2]
    
    if iterations == 0:
        print("Creating model")
    #Shared parameters for the models
    optimizer = Adam(lr=0, beta_1=0.5)
    lossFunction = "mean_squared_error"
    metrics = ["mse"]
    
    #Define the model
    nFeatures = len(selected_features)
    shapeSN = nFeatures*windows_size
    modelRULSN = RULmodel_SN_1(shapeSN)
    modelRULSN.compile(optimizer = optimizer, loss = lossFunction, metrics = metrics)

    #load the data using the selected parameters
    tModel = TunableModel(selected_features, scaler = min_max_scaler, window_stride = window_stride, 
                          window_size = windows_size, constRul = constantRUL, datasetNumber = datasetNumber, 
                         epochs=epochs)
    
    tModel.loadData()
    
    #Add the models to the tunableModel object
    tModel.addModel("RUL_SN_1_optmizable", modelRULSN)
    
    if iterations == 0:
        print("Training model")
    #Train the model
    lrate = LearningRateScheduler(CMAPSAuxFunctions.step_decay)
    tModel.trainCurrentModel(learningRateScheduler=lrate, verbose=verbose)
    time = tModel.getModelTimes(tModel.currentModelName)
    
    if iterations == 0:
        print("Training time {}".format(time))
    
    if iterations == 0:
        print("Assesing model performance")
    #Assess the model performance
    tModel.evaluateCurrentModel(["rhs"], crossValidation=True)
    cScores = tModel.getModelScores(tModel.currentModelName)
    rmse = math.sqrt(cScores['score_1'])
    rhs = cScores['rhs']
    #print("The score for this model is: {}".format(rmse))
    
    msgStr = "The model variables are " + str(x) + "\tThe scores are: [RMSE:{:.4f}, RHS:{:.4f}]\n".format(rmse, rhs)
    
    if saveToFile is not None:
        #print(msgStr)
        saveToFile.write(msgStr)
    else:
        print(msgStr)
    
    #Return RMSE as the performance metric to steer the search
    return rmse